In [1]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

In [3]:
text = open('input.txt', 'r', encoding='utf-8').read()
print("length of dataset in characters: ", len(text))
print(text[:200])

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [4]:
chars = sorted(list(set(text)))
# create a mapping from characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [5]:
block_size = 8
n_emb = 32
vocab_size = len(chars)
batch_size = 64
eval_iters = 200
eval_interval = 5000
data = torch.tensor(encode(text), dtype=torch.long)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n = int(0.9*len(data))

In [6]:
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [90]:
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_emb, head_size, bias=False)
        self.query = nn.Linear(n_emb, head_size, bias=False)
        self.value = nn.Linear(n_emb, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size )))
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x) # (B,T,C)
        q = self.query(x) # (B,T,C)
        v = self.value(x) # (B,T,C)
        # compute attention scores, mask it and normalize.
        score = q @ k.transpose(-2,-1) * (C**-0.5)# (B,T,C) @ (B,C,T) -> (B,T,T)
        score = score.masked_fill(self.tril[:T,:T]==0, float('-inf')) # (B,T,T)
        score = F.softmax(score, dim=1)
        # perform the weighted aggregation of the values
        out = score @ v # (B,T,T) @ (B,T,C) -> (B,T,C)
        return out

class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self, x):
      return torch.cat([head(x) for head in self.heads], dim=1)  # (B,T,C)


In [105]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_embedding_table = nn.Embedding(block_size, n_emb)
        # self.sa_heads = MultiHeadAttention(4, n_emb//4) # i.e 4 heads of 8-dim self attention
        self.sa_heads = Head(n_emb)
        self.lm_head = nn.Linear(n_emb, vocab_size)
        
        
    def forward(self, idx, targets=None):
        B , T = idx.shape
        token_emb = self.token_embedding_table(idx) #(B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #(T,C)
        x = token_emb + pos_emb  #(B,T,C)
        x = self.sa_heads(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B , T , C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits , loss
      
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:,-block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:,-1,:] #focus on only the last time step
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx,idx_next), dim=1) #append new sampled index to the running context
        return idx

In [106]:
model = BigramLanguageModel()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

In [108]:
max_iter = 10000
for iter in range(max_iter):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb , yb = get_batch('train')
    logit , loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.0781, val loss 4.0848
step 5000: train loss 2.1013, val loss 2.1212


In [10]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=100)[0].tolist()))

# saving and loading for inference 

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [12]:
# torch.save(model.state_dict(), PATH)

# # loading

# model = BigramLanguageModel() #instantiate model arcitecture
# model.load_state_dict(torch.load(PATH))

# saving and loading to continue training

In [13]:
# # saving
# torch.save({'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict()}, PATH)

# # loading
# checkpoint = torch.load(PATH, map_location=device)
# model = BigramLanguageModel() #instantiate model arcitecture
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3) #instantiate optimizer
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])